In [7]:
from pyforest import*

from sklearn.cluster import KMeans
from scipy.stats import zscore
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from scipy.stats import zscore
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA

In [8]:
data=pd.read_csv('abalone.csv',header=None,names=['Gender','Length','Diameter','Height','Meatweight','Weight','Gutweight','Shellweight','Rings'])

In [9]:
data.head()
df=data.drop('Gender',axis=1)
df.head()

,Length,Diameter,Height,Meatweight,Weight,Gutweight,Shellweight,Rings
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [10]:
df.corr()

,Length,Diameter,Height,Meatweight,Weight,Gutweight,Shellweight,Rings
Length,1.000000,0.986812,0.827554,0.925261,0.897914,0.903018,0.897706,0.556720
Diameter,0.986812,1.000000,0.833684,0.925452,0.893162,0.899724,0.905330,0.574660
Height,0.827554,0.833684,1.000000,0.819221,0.774972,0.798319,0.817338,0.557467
Meatweight,0.925261,0.925452,0.819221,1.000000,0.969405,0.966375,0.955355,0.540390
Weight,0.897914,0.893162,0.774972,0.969405,1.000000,0.931961,0.882617,0.420884
Gutweight,0.903018,0.899724,0.798319,0.966375,0.931961,1.000000,0.907656,0.503819
Shellweight,0.897706,0.905330,0.817338,0.955355,0.882617,0.907656,1.000000,0.627574
Rings,0.556720,0.574660,0.557467,0.540390,0.420884,0.503819,0.627574,1.000000


In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

df_scaled = df.apply(zscore) 
df_scaled.head()

,Length,Diameter,Height,Meatweight,Weight,Gutweight,Shellweight,Rings
0,-0.574558,-0.432149,-1.064424,-0.641898,-0.607685,-0.726212,-0.638217,1.571544
1,-1.448986,-1.439929,-1.183978,-1.230277,-1.170910,-1.205221,-1.212987,-0.910013
2,0.050033,0.122130,-0.107991,-0.309469,-0.463500,-0.356690,-0.207139,-0.289624
3,-0.699476,-0.432149,-0.347099,-0.637819,-0.648238,-0.607600,-0.602294,0.020571
4,-1.615544,-1.540707,-1.423087,-1.272086,-1.215968,-1.287337,-1.320757,-0.910013


In [12]:
cov_matrix=np.cov(df_scaled.T) #covariance convert row v/s row but we need column v/s column so we transpose it
print(cov_matrix)
print('shape==',cov_matrix.shape)

[[1.00023946 0.98704789 0.82775178 0.92548274 0.89812868 0.90323394
  0.89792054 0.55685289]
 [0.98704789 1.00023946 0.83388332 0.92567371 0.89337636 0.89993988
  0.90554657 0.57479746]
 [0.82775178 0.83388332 1.00023946 0.81941695 0.77515787 0.79851047
  0.81753374 0.55760082]
 [0.92548274 0.92567371 0.81941695 1.00023946 0.96963759 0.96660649
  0.95558422 0.54051908]
 [0.89812868 0.89337636 0.77515787 0.96963759 1.00023946 0.93218449
  0.88282841 0.42098444]
 [0.90323394 0.89993988 0.79851047 0.96660649 0.93218449 1.00023946
  0.90787367 0.5039399 ]
 [0.89792054 0.90554657 0.81753374 0.95558422 0.88282841 0.90787367
  1.00023946 0.62772433]
 [0.55685289 0.57479746 0.55760082 0.54051908 0.42098444 0.5039399
  0.62772433 1.00023946]]
shape== (8, 8)


In [13]:
eig_vals,eig_vecs = np.linalg.eig(cov_matrix)
eig_vals

array([6.71404654e+00, 6.95779541e-01, 2.58505008e-01, 1.66029591e-01,
       1.26972036e-02, 6.39985662e-03, 6.34879663e-02, 8.49700060e-02])

In [14]:
eig_vecs  # give row wise data 

array([[ 3.72138505e-01,  6.82826958e-02,  3.10703159e-02,
        -6.04053517e-01,  6.98825733e-01, -1.63485531e-02,
        -4.74968278e-02,  1.11248478e-02],
       [ 3.73094111e-01,  4.00480412e-02,  4.10050673e-02,
        -5.87594701e-01, -7.12985166e-01,  2.19254893e-04,
        -2.33754046e-02, -5.79110156e-02],
       [ 3.40026777e-01, -7.04631477e-02,  8.99705624e-01,
         2.56777039e-01,  8.61445194e-03, -2.68817140e-03,
         2.66914622e-02,  5.67292650e-02],
       [ 3.78307478e-01,  1.37346192e-01, -2.06193609e-01,
         2.41848954e-01, -8.33128844e-03, -8.50264371e-01,
         1.17255046e-01, -1.56556128e-02],
       [ 3.62454487e-01,  2.98839921e-01, -2.08286301e-01,
         1.83245659e-01, -9.28176619e-03,  3.91100542e-01,
         6.24892864e-01,  3.98525300e-01],
       [ 3.68557801e-01,  1.72978517e-01, -1.97379538e-01,
         2.65220563e-01, -2.73455394e-02,  2.04179031e-01,
        -7.65843810e-01,  3.09821489e-01],
       [ 3.70757823e-01, -4.540040

In [15]:
eig_vecs[:,0]   #first column data from (8*8 matrix)

array([0.3721385 , 0.37309411, 0.34002678, 0.37830748, 0.36245449,
       0.3685578 , 0.37075782, 0.24271277])

In [16]:
eig_vecs[:,1]   #first column data from (8*8 matrix)

array([ 0.0682827 ,  0.04004804, -0.07046315,  0.13734619,  0.29883992,
        0.17297852, -0.0454004 , -0.92120385])

In [17]:
sorted(eig_vals,reverse=True)

[6.714046535594113,
 0.6957795408508023,
 0.2585050082948898,
 0.1660295914776441,
 0.08497000602295975,
 0.06348796631816002,
 0.01269720363809055,
 0.006399856615600904]

In [18]:
6.7140/np.sum(eig_vals)

0.8390490782858511

In [19]:
tot=sum(eig_vals)
var_exp= [(i /tot) * 100 for i in sorted  (eig_vals , reverse= True)]    # percentage of variance explained
cum_var_exp= np.cumsum(var_exp)  #cumulative sum
print('cumulative varianceexplained ==', cum_var_exp)

# as high multicollinearity here so we can easily sacrify 97,98,99,99 for better model or reduce the dimensions

cumulative varianceexplained == [ 83.90548938  92.60065147  95.83119048  97.90606351  98.96793431
  99.7613439   99.92002094 100.        ]


In [20]:
eig_vals

array([6.71404654e+00, 6.95779541e-01, 2.58505008e-01, 1.66029591e-01,
       1.26972036e-02, 6.39985662e-03, 6.34879663e-02, 8.49700060e-02])

In [21]:
eigen_pairs= [(np.abs(eig_vals[i]), eig_vecs [: ,i]) for i in range (len(eig_vals))]



eigen_pairs          #convert into 2d list

[(6.714046535594113,
  array([0.3721385 , 0.37309411, 0.34002678, 0.37830748, 0.36245449,
         0.3685578 , 0.37075782, 0.24271277])),
 (0.6957795408508023,
  array([ 0.0682827 ,  0.04004804, -0.07046315,  0.13734619,  0.29883992,
          0.17297852, -0.0454004 , -0.92120385])),
 (0.2585050082948898,
  array([ 0.03107032,  0.04100507,  0.89970562, -0.20619361, -0.2082863 ,
         -0.19737954, -0.16157408, -0.19214359])),
 (0.1660295914776441,
  array([-0.60405352, -0.5875947 ,  0.25677704,  0.24184895,  0.18324566,
          0.26522056,  0.24419207,  0.04331013])),
 (0.01269720363809055,
  array([ 0.69882573, -0.71298517,  0.00861445, -0.00833129, -0.00928177,
         -0.02734554,  0.04739508,  0.00842057])),
 (0.006399856615600904,
  array([-1.63485531e-02,  2.19254893e-04, -2.68817140e-03, -8.50264371e-01,
          3.91100542e-01,  2.04179031e-01,  2.85623992e-01,  2.33703940e-02])),
 (0.06348796631816002,
  array([-0.04749683, -0.0233754 ,  0.02669146,  0.11725505,  0.62489

In [22]:
eigen_pairs[2][1]

array([ 0.03107032,  0.04100507,  0.89970562, -0.20619361, -0.2082863 ,
       -0.19737954, -0.16157408, -0.19214359])

In [23]:
eigen_pairs_sorted = sorted(eigen_pairs ,reverse=True)
eigen_pairs_sorted

[(6.714046535594113,
  array([0.3721385 , 0.37309411, 0.34002678, 0.37830748, 0.36245449,
         0.3685578 , 0.37075782, 0.24271277])),
 (0.6957795408508023,
  array([ 0.0682827 ,  0.04004804, -0.07046315,  0.13734619,  0.29883992,
          0.17297852, -0.0454004 , -0.92120385])),
 (0.2585050082948898,
  array([ 0.03107032,  0.04100507,  0.89970562, -0.20619361, -0.2082863 ,
         -0.19737954, -0.16157408, -0.19214359])),
 (0.1660295914776441,
  array([-0.60405352, -0.5875947 ,  0.25677704,  0.24184895,  0.18324566,
          0.26522056,  0.24419207,  0.04331013])),
 (0.08497000602295975,
  array([ 0.01112485, -0.05791102,  0.05672927, -0.01565561,  0.3985253 ,
          0.30982149, -0.83056377,  0.22002569])),
 (0.06348796631816002,
  array([-0.04749683, -0.0233754 ,  0.02669146,  0.11725505,  0.62489286,
         -0.76584381,  0.03283235,  0.06819579])),
 (0.01269720363809055,
  array([ 0.69882573, -0.71298517,  0.00861445, -0.00833129, -0.00928177,
         -0.02734554,  0.047

In [24]:
eigen_pairs_sorted[1][1][5]  #fetching values

0.17297851676041756

In [25]:
eig_vals

array([6.71404654e+00, 6.95779541e-01, 2.58505008e-01, 1.66029591e-01,
       1.26972036e-02, 6.39985662e-03, 6.34879663e-02, 8.49700060e-02])

In [32]:
## Extract descending ordered eigenvalues and eigen vectors
eigenvalues_sort= [ eigen_pairs_sorted [index][0] for index in range (len(eig_vals))]
eigenvectors_sort= [ eigen_pairs_sorted [index][1] for index in range (len(eig_vals))]

#project data onto 2d
eigenvectors_sort      #again give output row wise here

[array([0.3721385 , 0.37309411, 0.34002678, 0.37830748, 0.36245449,
        0.3685578 , 0.37075782, 0.24271277]),
 array([ 0.0682827 ,  0.04004804, -0.07046315,  0.13734619,  0.29883992,
         0.17297852, -0.0454004 , -0.92120385]),
 array([ 0.03107032,  0.04100507,  0.89970562, -0.20619361, -0.2082863 ,
        -0.19737954, -0.16157408, -0.19214359]),
 array([-0.60405352, -0.5875947 ,  0.25677704,  0.24184895,  0.18324566,
         0.26522056,  0.24419207,  0.04331013]),
 array([ 0.01112485, -0.05791102,  0.05672927, -0.01565561,  0.3985253 ,
         0.30982149, -0.83056377,  0.22002569]),
 array([-0.04749683, -0.0233754 ,  0.02669146,  0.11725505,  0.62489286,
        -0.76584381,  0.03283235,  0.06819579]),
 array([ 0.69882573, -0.71298517,  0.00861445, -0.00833129, -0.00928177,
        -0.02734554,  0.04739508,  0.00842057]),
 array([-1.63485531e-02,  2.19254893e-04, -2.68817140e-03, -8.50264371e-01,
         3.91100542e-01,  2.04179031e-01,  2.85623992e-01,  2.33703940e-02])]

In [33]:
eigenvalues_sort

[6.714046535594113,
 0.6957795408508023,
 0.2585050082948898,
 0.1660295914776441,
 0.08497000602295975,
 0.06348796631816002,
 0.01269720363809055,
 0.006399856615600904]

In [28]:
eigenvectors_sort[0:4]

[array([0.3721385 , 0.37309411, 0.34002678, 0.37830748, 0.36245449,
        0.3685578 , 0.37075782, 0.24271277]),
 array([ 0.0682827 ,  0.04004804, -0.07046315,  0.13734619,  0.29883992,
         0.17297852, -0.0454004 , -0.92120385]),
 array([ 0.03107032,  0.04100507,  0.89970562, -0.20619361, -0.2082863 ,
        -0.19737954, -0.16157408, -0.19214359]),
 array([-0.60405352, -0.5875947 ,  0.25677704,  0.24184895,  0.18324566,
         0.26522056,  0.24419207,  0.04331013])]

In [29]:

## keep the first four principal components
## P_reduce is 8*4 matrix

p_reduce= np.array(eigenvectors_sort[0:4]).transpose()
p_reduce

array([[ 0.3721385 ,  0.0682827 ,  0.03107032, -0.60405352],
       [ 0.37309411,  0.04004804,  0.04100507, -0.5875947 ],
       [ 0.34002678, -0.07046315,  0.89970562,  0.25677704],
       [ 0.37830748,  0.13734619, -0.20619361,  0.24184895],
       [ 0.36245449,  0.29883992, -0.2082863 ,  0.18324566],
       [ 0.3685578 ,  0.17297852, -0.19737954,  0.26522056],
       [ 0.37075782, -0.0454004 , -0.16157408,  0.24419207],
       [ 0.24271277, -0.92120385, -0.19214359,  0.04331013]])

In [30]:
## project data ONTO 2D space

proj_data_pca = np.dot(df_scaled, p_reduce)
print(proj_data_pca)



[[-1.32291442 -1.79565502 -0.78981594 -0.21931559]
 [-3.48365134  0.09283156 -0.06301054  0.24997171]
 [-0.53616125  0.04940728  0.22927472 -0.44722137]
 ...
 [ 2.03301641  0.69391768  0.9370173   0.29820072]
 [ 1.60123963  0.46575061 -0.20441808 -0.33186727]
 [ 4.89861662  0.80973552 -0.49177154  0.52127292]]


In [31]:
print('fetch=',proj_data_pca[0,:])

fetch= [-1.32291442 -1.79565502 -0.78981594 -0.21931559]


In [65]:
## modeling

model=KMeans(n_clusters=3, n_init=15,random_state=0)
model.fit(proj_data_pca)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=15, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [67]:
model.cluster_centers_  # for 1 label there is nice separation so go for the 1label onmly

array([[-3.21280833,  0.11121902, -0.07441783,  0.15687787],
       [ 2.97542453,  0.14251892, -0.08163922,  0.1472999 ],
       [-0.00847287, -0.17734705,  0.10873678, -0.21128029]])

In [68]:
## create another model 


## keep the first four principal components
## P_reduce is 8*4 matrix

p_reduce= np.array(eigenvectors_sort[0:1]).transpose()
p_reduce

## project data ONTO 2D space

proj_data_pca = np.dot(df_scaled, p_reduce)
print(proj_data_pca)

## modeling

model=KMeans(n_clusters=3, n_init=15,random_state=0)
model.fit(proj_data_pca)

model.cluster_centers_

[[-1.32291442]
 [-3.48365134]
 [-0.53616125]
 ...
 [ 2.03301641]
 [ 1.60123963]
 [ 4.89861662]]


array([[-3.11714358],
       [ 3.09118864],
       [ 0.13531779]])

In [70]:
L=pd.DataFrame(model.labels_)
L[0].value_counts()

2    1766
0    1237
1    1174
Name: 0, dtype: int64

In [71]:
data.Gender.value_counts()

M    1528
I    1342
F    1307
Name: Gender, dtype: int64

In [72]:
model.inertia_

4786.876980301662

In [76]:
from sklearn import metrics
metrics.silhouette_score(df_scaled,model.labels_)

0.38407428104236585

In [77]:
df_scaled_labeled=df_scaled.copy()
df_scaled_labeled['class']=model.labels_

In [78]:
df_scaled_labeled.head()

,Length,Diameter,Height,Meatweight,Weight,Gutweight,Shellweight,Rings,class
0,-0.574558,-0.432149,-1.064424,-0.641898,-0.607685,-0.726212,-0.638217,1.571544,2
1,-1.448986,-1.439929,-1.183978,-1.230277,-1.170910,-1.205221,-1.212987,-0.910013,0
2,0.050033,0.122130,-0.107991,-0.309469,-0.463500,-0.356690,-0.207139,-0.289624,2
3,-0.699476,-0.432149,-0.347099,-0.637819,-0.648238,-0.607600,-0.602294,0.020571,2
4,-1.615544,-1.540707,-1.423087,-1.272086,-1.215968,-1.287337,-1.320757,-0.910013,0


In [81]:

### AGC_clusters= data_scaled_df_labeled.groupby(['class'])

AGC_clusters= df_scaled_labeled.groupby(['class'])


Df0 = AGC_clusters.get_group(0)
Df1 = AGC_clusters.get_group(1)
Df2 = AGC_clusters.get_group(2)


Df0.describe()

### 0 class as infants
### 1 class as female
### 2 class as male


from sklearn.cluster import AgglomerativeClustering
m2=AgglomerativeClustering(n_clusters=3, affinity='manhattan' ,linkage='single')
m2.fit(df_scaled_update)


L=pd.DataFrame(m2.labels_)
L[0].value_counts()           # from all choices ward method is better 
print(L[0].value_counts())

##Adding a label feature with the predicted class values:
data_scaled_df_labeled= df_scaled_update.copy()
data_scaled_df_labeled['class']=m2.labels_
data_scaled_df_labeled.head()



### AGC_clusters= data_scaled_df_labeled.groupby(['class'])

AGC_clusters= data_scaled_df_labeled.groupby(['class'])


df0 = AGC_clusters.get_group(0)
df1 = AGC_clusters.get_group(1)
df2 = AGC_clusters.get_group(2)


c0=np.array(df0.mean())
c1=np.array(df1.mean())
c2=np.array(df2.mean())


c0=c0[:-1] #as last one record is for label
c1=c1[:-1]
c2=c2[:-1]

AGC_I0=0
AGC_I1=0
AGC_I2=0

for i in np.arange(df0.shape[0]):
    AGC_I0=AGC_I0+np.sum((df0.iloc[i,:-1]-c0)**2)       # bcoz npo require label column so we use iloc[i,:-1]
for i in np.arange(df1.shape[0]):
    AGC_I1=AGC_I1+np.sum((df1.iloc[i,:-1]-c1)**2)
for i in np.arange(df2.shape[0]):
    AGC_I2=AGC_I2+np.sum((df2.iloc[i,:-1]-c2)**2)
    
AGC_I0,AGC_I1,AGC_I2


from sklearn import metrics

print(metrics.silhouette_score(df_scaled_update,m2.labels_))



### from this there is only single information we will get i.e heterogenity bcoz from formula a is going to be zero ((b-a)/max(a,b))

,Length,Diameter,Height,Meatweight,Weight,Gutweight,Shellweight,Rings,class
count,1237.000000,1237.000000,1237.000000,1237.000000,1237.000000,1237.000000,1237.000000,1237.000000,1237.0
mean,-1.239880,-1.241510,-1.088876,-1.117945,-1.065741,-1.097319,-1.106923,-0.822246,0.0
std,0.658735,0.646045,0.545383,0.287938,0.307384,0.285927,0.306433,0.609819,0.0
min,-3.739154,-3.556267,-3.335953,-1.686092,-1.614731,-1.643173,-1.705134,-2.771180,0.0
25%,-1.615544,-1.591096,-1.423087,-1.340407,-1.301578,-1.319271,-1.342311,-1.220208,0.0
50%,-1.115871,-1.137595,-1.064424,-1.118108,-1.085300,-1.109419,-1.105218,-0.910013,0.0
75%,-0.699476,-0.734483,-0.705762,-0.878473,-0.850998,-0.872195,-0.864533,-0.599818,0.0
max,0.008394,0.273297,0.489780,-0.369633,0.611132,-0.197020,0.802301,2.502127,0.0


In [82]:
Df1.describe()

,Length,Diameter,Height,Meatweight,Weight,Gutweight,Shellweight,Rings,class
count,1174.000000,1174.000000,1174.000000,1174.000000,1174.000000,1174.000000,1174.000000,1174.000000,1174.0
mean,1.051330,1.058791,1.003944,1.242832,1.190682,1.220012,1.205094,0.665797,1.0
std,0.357366,0.355691,0.841018,0.624910,0.735679,0.675273,0.689268,0.974818,0.0
min,-0.574558,-0.532927,-0.347099,-0.478743,-0.267498,-0.589352,-0.756763,-1.220208,1.0
25%,0.799543,0.827576,0.609334,0.793869,0.689984,0.742751,0.720576,0.020571,1.0
50%,1.007740,1.029133,0.848442,1.086528,1.068471,1.107711,1.069929,0.330765,1.0
75%,1.257576,1.281078,1.207105,1.563248,1.507786,1.608390,1.538726,0.951154,1.0
max,2.423480,2.440025,23.683287,4.072271,5.085388,5.286500,5.504642,5.914268,1.0


In [83]:
Df2.describe()

,Length,Diameter,Height,Meatweight,Weight,Gutweight,Shellweight,Rings,class
count,1766.000000,1766.000000,1766.000000,1766.000000,1766.000000,1766.000000,1766.000000,1766.000000,1766.0
mean,0.169575,0.165757,0.095305,-0.043141,-0.045039,-0.042418,-0.025774,0.133337,2.0
std,0.369376,0.369868,0.421595,0.379178,0.446822,0.412958,0.399627,0.829538,0.0
min,-0.990952,-0.986428,-2.977291,-0.857059,-1.103323,-1.040989,-1.001041,-1.220208,2.0
25%,-0.116525,-0.129815,-0.227545,-0.366064,-0.389154,-0.361252,-0.350832,-0.289624,2.0
50%,0.174951,0.172519,0.131117,-0.040263,-0.081634,-0.073846,-0.027523,0.020571,2.0
75%,0.466427,0.474853,0.370226,0.254436,0.279956,0.259180,0.259862,0.330765,2.0
max,1.091018,1.180300,2.283092,1.247644,1.566361,1.294753,1.732711,4.053100,2.0


### Apply PCA by code:--



pca = PCA(n_components = 2) 
  
X_train = pca.fit_transform(X_train) 
X_test = pca.transform(X_test) 
  
explained_variance = pca.explained_variance_ratio_ 